# Preprocessing BOK call rates 

한국은행에서 다운받은 call rates 파일을 전처리한다. 
- 빈칸은 ffill
- string object인 date를 datetime 화. 
- 30일 전 콜금리와 비교하여 올랐는지 내렸는지 동일한지 표기 

In [36]:
import pandas as pd
import numpy as np

In [37]:
df = pd.read_csv('./data_files/BOK_4.1.1_market_call_rate_-_daily.csv', encoding='utf-8')
df.head()

,date,call_rate
0,1/1/2004,NaN
1,1/2/2004,3.77
2,1/3/2004,NaN
3,1/4/2004,NaN
4,1/5/2004,3.76


In [38]:
df = df.fillna(method='ffill')

In [39]:
df.date = pd.to_datetime(df.date, format='%m/%d/%Y')

In [40]:
df = df.merge(df.assign(date = df.date+pd.Timedelta(days=30)), 
               on='date',
               how='left', suffixes=['', '_30daysago'])

In [41]:
df.dropna(inplace=True)

In [42]:
df['change'] = np.select(
    condlist=[
        (df.call_rate > df. call_rate_30daysago), 
        (df.call_rate == df.call_rate_30daysago),
    ], 
    choicelist=[
        1,
        0,
    ],
    default=-1
)

In [43]:
df

,date,call_rate,call_rate_30daysago,change
31,2004-02-01,3.78,3.77,1
32,2004-02-02,3.78,3.77,1
33,2004-02-03,3.77,3.77,0
34,2004-02-04,3.78,3.76,1
35,2004-02-05,3.79,3.74,1
...,...,...,...,...
5890,2020-02-16,1.24,1.28,-1
5891,2020-02-17,1.24,1.28,-1
5892,2020-02-18,1.23,1.28,-1
5893,2020-02-19,1.24,1.28,-1


In [44]:
df.to_pickle('./data_files/BOK_call_rates.pkl')